## 0. Install and Import Dependencies

In [7]:
# pip install selenium
# pip install webdriver-manager

In [38]:
import selenium
print(selenium.__version__)
import requests
print(requests.__version__)

4.36.0
2.31.0


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import requests
import time
import os
import json
import urllib3
from tqdm import tqdm

## 1. Get video data (id, word label, url)

In [29]:
# Setup and Access
options = Options()
options.add_argument("--headless")
options.add_argument("--ignore-certificate-errors")
options.add_argument("--allow-insecure-localhost")
options.add_argument("--ignore-ssl-errors=yes")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

BASE_URL = "https://qipedc.moet.gov.vn"
URL = "https://qipedc.moet.gov.vn/dictionary"
try:
    driver.get(URL)
    driver.implicitly_wait(5)
    print(f"Sucessfully connected to {driver.title} website with url {driver.current_url}.")
    videos = driver.find_elements(By.CSS_SELECTOR, "#product a")
    print(f"Found {len(videos)} videos.")
    video_data = []
    start_time = time.time()
    for i, vid in enumerate(videos):
        try:
            # get word label
            label = vid.find_element(By.TAG_NAME, "p").text.strip()
            thumbs_url = vid.find_element(By.CSS_SELECTOR, "img").get_attribute("src")
            vid_id = thumbs_url.replace("https://qipedc.moet.gov.vn/thumbs/", "").replace(".png", "")
            video_url = f"{BASE_URL}/videos/{vid_id}.mp4"
            # vid_data = onclick.split("'")
            # vid_id = vid_data[1]
            # vid_definition = vid_data[5]
            
            # onclick = vid.get_attribute("onclick")
            # driver.execute_script(onclick)
            # WebDriverWait(driver, 5).until(
            #     expected_conditions.presence_of_element_located((By.CSS_SELECTOR, "#s_expert"))
            # )
            # iframe = driver.find_element(By.CSS_SELECTOR, "#s_expert")
            # video_url = iframe.get_attribute("src")
            # video_url = video_url.replace('?autoplay=true','')
            # print(video_url)
            
            video_data.append({
                "id": vid_id,
                "word": label,
                "video_url": video_url
            })
            
            # driver.execute_script("$('#exampleModal').modal('hide');")
            # close_btn = driver.find_element(By.CSS_SELECTOR, "button[data-dismiss='modal']")
            # close_btn.click()
            time.sleep(0.5)
        except Exception as e:
            print(f"Error at video {i}: {e}")
    print(f"Sucessfully get {len(videos)} video data in {time.time() - start_time}s.")
        
except Exception as e:
    print(f"Error: {e}")
finally:
    driver.quit()

Sucessfully connected to QIPEDC website with url https://qipedc.moet.gov.vn/dictionary.
Found 20 videos.
Sucessfully get 20 video data in 10.913319826126099s.


In [30]:
video_data

[{'id': 'D0001B',
  'word': 'địa chỉ',
  'video_url': 'https://qipedc.moet.gov.vn/videos/D0001B.mp4'},
 {'id': 'D0001N',
  'word': 'địa chỉ',
  'video_url': 'https://qipedc.moet.gov.vn/videos/D0001N.mp4'},
 {'id': 'D0001T',
  'word': 'địa chỉ',
  'video_url': 'https://qipedc.moet.gov.vn/videos/D0001T.mp4'},
 {'id': 'D0002',
  'word': 'tỉnh',
  'video_url': 'https://qipedc.moet.gov.vn/videos/D0002.mp4'},
 {'id': 'D0003',
  'word': 'tiếp tân',
  'video_url': 'https://qipedc.moet.gov.vn/videos/D0003.mp4'},
 {'id': 'D0004',
  'word': 'nhân viên',
  'video_url': 'https://qipedc.moet.gov.vn/videos/D0004.mp4'},
 {'id': 'D0005B',
  'word': 'Miến Điện (nước Mi-an-ma)',
  'video_url': 'https://qipedc.moet.gov.vn/videos/D0005B.mp4'},
 {'id': 'D0005N',
  'word': 'Miến Điện (nước Mi-an-ma)',
  'video_url': 'https://qipedc.moet.gov.vn/videos/D0005N.mp4'},
 {'id': 'D0005T',
  'word': 'Miến Điện (nước Mi-an-ma)',
  'video_url': 'https://qipedc.moet.gov.vn/videos/D0005T.mp4'},
 {'id': 'D0006',
  'word'

## 2. Download videos from video url

In [36]:
video_dir = 'dataset/videos'
chunk_size = 1024*32

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

os.makedirs(video_dir, exist_ok=True)
for item in video_data:
    vid_url = item['video_url']
    vid_id = item['id']
    filename = f"{vid_id}.mp4"
    output_path = os.path.join(video_dir, filename)
    
    if os.path.exists(output_path):
        print(output_path)
        continue
    response = requests.get(vid_url, stream=True, verify=False, timeout=10)
    response.raise_for_status()
    total_size = int(response.headers.get('content-length', 0))
    
    with open(output_path, 'wb') as f, tqdm(
        total=total_size, unit='B', unit_scale=True, desc=f"Progess {filename}"
    ) as bar:
        for chunk in response.iter_content(chunk_size=chunk_size):
            if chunk:
                f.write(chunk)
                bar.update(len(chunk))

Progess D0001B.mp4: 100%|██████████| 686k/686k [00:00<00:00, 1.03MB/s]
Progess D0001N.mp4: 100%|██████████| 595k/595k [00:00<00:00, 1.23MB/s]
Progess D0001T.mp4: 100%|██████████| 585k/585k [00:00<00:00, 1.41MB/s]
Progess D0002.mp4: 100%|██████████| 1.00M/1.00M [00:00<00:00, 1.70MB/s]
Progess D0003.mp4: 100%|██████████| 560k/560k [00:00<00:00, 1.06MB/s]
Progess D0004.mp4: 100%|██████████| 678k/678k [00:01<00:00, 394kB/s] 
Progess D0005B.mp4: 100%|██████████| 876k/876k [00:01<00:00, 800kB/s] 
Progess D0005N.mp4: 100%|██████████| 1.06M/1.06M [00:00<00:00, 1.47MB/s]
Progess D0005T.mp4: 100%|██████████| 1.07M/1.07M [00:00<00:00, 1.78MB/s]
Progess D0006.mp4: 100%|██████████| 577k/577k [00:00<00:00, 1.37MB/s]
Progess D0007.mp4: 100%|██████████| 781k/781k [00:01<00:00, 699kB/s] 
Progess D0008.mp4: 100%|██████████| 459k/459k [00:00<00:00, 1.03MB/s]
Progess D0009.mp4: 100%|██████████| 380k/380k [00:00<00:00, 995kB/s] 
Progess D0010.mp4: 100%|██████████| 536k/536k [00:00<00:00, 1.35MB/s]
Progess 

In [ ]:
from IPython.display import Video, display

display(Video("dataset/videos/D0001B.mp4", embed=True, width=800, height=450))

## 3. Turn page

In [ ]:
total_pages = 10

for page_num in range(1, total_pages + 1):
    try:
        WebDriverWait(driver, 5).until(
            expected_conditions.presence_of_element_located((By.CSS_SELECTOR, "#pagination-wrapper"))
        )

        buttons = driver.find_elements(By.CSS_SELECTOR, "#pagination-wrapper button.page")
        for btn in buttons:
            if btn.get_attribute("value") == str(page_num):
                driver.execute_script("arguments[0].click();", btn)
                time.sleep(1.5)

                current_page = driver.find_element(By.CSS_SELECTOR, "#pagination-wrapper .btn-info")
                print(f"Current page: {current_page.get_attribute('value')}")
                break

    except Exception as e:
        print(f"Cannot go to page {page_num}: {e}")


Không thể chuyển trang 1: HTTPConnectionPool(host='localhost', port=55623): Max retries exceeded with url: /session/40266977e0a8f45e1eca8ae81f32a4b6/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x116a83f10>: Failed to establish a new connection: [Errno 61] Connection refused'))
Không thể chuyển trang 2: HTTPConnectionPool(host='localhost', port=55623): Max retries exceeded with url: /session/40266977e0a8f45e1eca8ae81f32a4b6/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x116a49850>: Failed to establish a new connection: [Errno 61] Connection refused'))
Không thể chuyển trang 3: HTTPConnectionPool(host='localhost', port=55623): Max retries exceeded with url: /session/40266977e0a8f45e1eca8ae81f32a4b6/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x116a4b250>: Failed to establish a new connection: [Errno 61] Connection refused'))
Không thể chuyển trang 4: HTTPConnectionPool(h